In [5]:
import torch
import time
from torch_geometric.utils import to_dense_adj
from torch_sparse import SparseTensor

In [6]:
edge_index = torch.tensor([[0,0,1,1,2,2],[1,2,3,4,4,5]], dtype=torch.long)  # 2 x E
x = torch.tensor([[1], [1], [1], [1], [1], [1]], dtype=torch.float)  # N x emb(in)
print("Original Edge_index: \n", edge_index)

def one_step_sparse(edge_index, x, num_nodes):
    edge_weight = torch.ones(edge_index.size(1))
    size = torch.Size([num_nodes, num_nodes])
    adj = torch.sparse_coo_tensor(edge_index, edge_weight, size = size)
    if x>1: 
        adj_powers = [adj]
        for k in range(2, x+1):
            adj_powers.append(torch.sparse.mm(adj_powers[k-2], adj_powers[0]))
            adj_k = adj_powers[k-1] - adj_powers[0]
            adj_k = torch.sparse_coo_tensor(adj_k._indices(), 1/k * (adj_k._values()>0)*1, size = size)
            adj = adj + adj_k   
    return SparseTensor.from_torch_sparse_coo_tensor(adj), None

avg_passed = 0
n= 0
for i in range(1):   
    start = time.time_ns()
    hop2, w2 = one_step_sparse(edge_index, 2, x.size(0))
    end = time.time_ns()
    passed = 1 * (end-start)
    if passed>0:
        n+=1
    avg_passed += passed

print()
print("Hop2-Sparse: \n", hop2)
print("Hop2-Dense: \n", hop2.to_dense())
print()
print("Avg. time: \n", avg_passed/n)

Original Edge_index: 
 tensor([[0, 0, 1, 1, 2, 2],
        [1, 2, 3, 4, 4, 5]])

Hop2-Sparse: 
 SparseTensor(row=tensor([0, 0, 0, 0, 0, 1, 1, 2, 2]),
             col=tensor([1, 2, 3, 4, 5, 3, 4, 4, 5]),
             val=tensor([1.0000, 1.0000, 0.5000, 0.5000, 0.5000, 1.0000, 1.0000, 1.0000, 1.0000]),
             size=(6, 6), nnz=9, density=25.00%)
Hop2-Dense: 
 tensor([[0.0000, 1.0000, 1.0000, 0.5000, 0.5000, 0.5000],
        [0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])

Avg. time: 
 998300.0


In [7]:
edge_index = torch.tensor([[0,0,1,1,2,2],[1,2,3,4,4,5]], dtype=torch.long)  # 2 x E
x = torch.tensor([[1], [1], [1], [1], [1], [1]], dtype=torch.float)  # N x emb(in)
print("Original Edge_index: \n", edge_index)

def one_step(edge_index, x):
    edge_weight = None
    if x>1: 
        edge_weight = torch.ones(edge_index.size(1))
        adj = to_dense_adj(edge_index)[0]
        for k in range(2,x+1):
            adj_k = (torch.linalg.matrix_power(adj, k)>0)*1 - adj
            edge_index_k = (adj_k > 0).nonzero().t()
            edge_weight_k = 1/k * torch.ones(edge_index_k.size(1))
            edge_index = torch.cat((edge_index, edge_index_k), dim=1)
            edge_weight = torch.cat((edge_weight, edge_weight_k), dim=0)

    return edge_index, edge_weight

avg_passed = 0
n= 0
for i in range(1000):   
    start = time.time_ns()
    hop2, w2 = one_step(edge_index, 2)
    end = time.time_ns()
    passed = 1 * (end-start)
    if passed>0:
        n+=1
    avg_passed += passed

print()
print("Hop2-Edge-Index:\n", hop2)
print("Edge-Weights:\n", w2)
print()
print("Avg. time:", avg_passed/n)

Original Edge_index: 
 tensor([[0, 0, 1, 1, 2, 2],
        [1, 2, 3, 4, 4, 5]])
tensor([[ 0., -1., -1.,  1.,  1.,  1.],
        [ 0.,  0.,  0., -1., -1.,  0.],
        [ 0.,  0.,  0.,  0., -1., -1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.]])

Hop2-Edge-Index:
 tensor([[0, 0, 1, 1, 2, 2, 0, 0, 0],
        [1, 2, 3, 4, 4, 5, 3, 4, 5]])
Edge-Weights:
 tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.5000, 0.5000, 0.5000])

Avg. time: 2993400.0
